# 1.Читаем данные

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

In [2]:
# Загрузка датасета
raw_data = pd.read_csv("C:/Users/Romas/Downloads/Loan.csv")
true_labels = (raw_data['LoanApproved']).to_numpy()
raw_data = raw_data.drop(['ApplicationDate', 'LoanApproved', 'RiskScore'], axis=1)

# Выделение численных признаков
numeric_feat = []
for feat in raw_data.keys():
  if ((raw_data.dtypes[feat] == 'float64' or raw_data.dtypes[feat] == 'int64') and feat != 'LoanApproved'):
      numeric_feat.append(feat)

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         20000 non-null  int64  
 1   AnnualIncome                20000 non-null  int64  
 2   CreditScore                 20000 non-null  int64  
 3   EmploymentStatus            20000 non-null  object 
 4   EducationLevel              20000 non-null  object 
 5   Experience                  20000 non-null  int64  
 6   LoanAmount                  20000 non-null  int64  
 7   LoanDuration                20000 non-null  int64  
 8   MaritalStatus               20000 non-null  object 
 9   NumberOfDependents          20000 non-null  int64  
 10  HomeOwnershipStatus         20000 non-null  object 
 11  MonthlyDebtPayments         20000 non-null  int64  
 12  CreditCardUtilizationRate   20000 non-null  float64
 13  NumberOfOpenCreditLines     200

# 2.Обрабатываем данные

In [3]:
# Преобразование категориальных признаков
for_use_data = pd.get_dummies(raw_data, columns=['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'], drop_first=True)

# Нормализация численных признаков
scaler = StandardScaler()
for_use_data[numeric_feat] = scaler.fit_transform(for_use_data[numeric_feat])
for_use_data = for_use_data.astype(float)

#  3. Кластеризация

##  3.1 KMeans

In [4]:
clustering = KMeans(n_clusters=2, random_state=0, n_init=10, tol=1e-8).fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_))} кластера")
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Корректность - {correct/len(true_labels)*100}%")
#print(class_correctness)

Обнаружено 2 кластера
12985 элементов имеют корректные предсказанные метки. Корректность - 64.925%


## 3.2 DBSCAN 

In [5]:
clustering = DBSCAN(eps=5.01, min_samples=8).fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_)) - (1) if ((-1) in clustering.labels_) else (0)} кластера.") # -1 - особая метка для "шума"
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Корректность - {correct/len(true_labels)*100}%")
#print(class_correctness)

Обнаружено 2 кластера.
13972 элементов имеют корректные предсказанные метки. Корректность - 69.86%


## 3.3 AgglomerativeClustering

In [6]:
clustering = AgglomerativeClustering().fit(for_use_data)

print(f"Обнаружено {len(set(clustering.labels_))} кластера.")
correct = 0
class_correctness = {}
for i in range(len(clustering.labels_)):
    if (clustering.labels_[i] == true_labels[i]):
        correct += 1
        class_correctness[clustering.labels_[i]] = (1) if not (clustering.labels_[i] in class_correctness.keys()) else (class_correctness[clustering.labels_[i]] + 1)
print(f"{correct} элементов имеют корректные предсказанные метки. Корректность - {correct/len(true_labels)*100}%")
#print(class_correctness)

Обнаружено 2 кластера.
14401 элементов имеют корректные предсказанные метки. Корректность - 72.005%


# 4. Сравнение методов

In [7]:
table1 = [["KMeans", 64.925],
         ["DBSCAN", 69.86],
         ["AgglomerativeClustering", 72.005]]

In [8]:
df = pd.DataFrame(table1, columns=["Метод", "Корректность в %"])

In [9]:
print(df)

                     Метод  Корректность в %
0                   KMeans            64.925
1                   DBSCAN            69.860
2  AgglomerativeClustering            72.005


# Вывод: Лучший результат получился с использованием метода кластеризации AgglomerativeClustering